# Vehicle Detection using HOG and SVM

In [ ]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import glob
import time
import numpy as np
import cv2
import math
import pickle
from skimage.feature import hog
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from scipy.ndimage.measurements import label
%matplotlib inline

##  Helper Function for feature extraction

In [ ]:
# While it could be cumbersome to include three color channels of a full resolution 
# image, you can perform spatial binning on an image and still retain enough information 
# to help in finding vehicles. 
def bin_spatial(img, size=(32, 32), print_flag = 'no'):
    """
    resize image to a desize size and 
    fatten out to a feature vector.
    """
    # Use cv2.resize().ravel() to create the feature vector
    features = cv2.resize(img, size).ravel() 
    if (print_flag == 'yes'):
        plt.plot(features)
        plt.title('Spatially Binned Features')
    # Return the feature vector
    return features


# Histogram of Color is useful in the case when objects appear slightly different in aspects and orientations
# are still be matched.
def color_hist(img, nbins=32, bins_range=(0, 256), print_flag = 'no'):
    """
    find color histogram for RGB channels and concatenate to a feature vector.
    Plot to visual R,G,B histogram graphs.
    """
    # Compute the histogram of the color channels separately
    rhist = np.histogram(img[:, :, 0], bins=nbins, range=bins_range)
    ghist = np.histogram(img[:, :, 1], bins=nbins, range=bins_range)
    bhist = np.histogram(img[:, :, 2], bins=nbins, range=bins_range)
    # Concatenate the histograms into a single feature vector
    hist_features = np.concatenate((rhist[0], ghist[0], bhist[0]))
    # Generating bin centers
    bin_edges = rhist[1]
    bin_centers = (bin_edges[1:] + bin_edges[0:len(bin_edges) - 1]) / 2
    # Plot a figure with all three bar charts
    if (print_flag == 'yes'):
        if (rhist is not None) or (ghist is not None) or (bhist is not None):
            fig = plt.figure(figsize=(12, 3))
            plt.subplot(131)
            plt.bar(bin_centers, rhist[0])
            plt.xlim(0, 256)
            plt.title('R Histogram')
            plt.subplot(132)
            plt.bar(bin_centers, ghist[0])
            plt.xlim(0, 256)
            plt.title('G Histogram')
            plt.subplot(133)
            plt.bar(bin_centers, bhist[0])
            plt.xlim(0, 256)
            plt.title('B Histogram')
            plt.show()
        else:
            print('Your function is returning None for at least one variable...')
    # Return the individual histograms, bin_centers and feature vector
    return hist_features




# HOG is good for distinguish gradient features
def get_hog_features(img, orient, pix_per_cell, cell_per_block, vis=False, feature_vec=True, print_flag='no'):
    """
    Given orientations, pixels_per_cell and cell_per_block, compute the hog features 
    of a image.
    If visual option is True, we can display the hog image on the graph.
    """
    
    # Call with two outputs if vis==True
    if vis == True:
        features, hog_image = hog(img, orientations=orient, 
                                  pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block), 
                                  transform_sqrt=False, 
                                  visualise=vis, feature_vector=feature_vec)
        if (print_flag == 'yes'):
            # Plot the examples
            fig = plt.figure()
            plt.subplot(121)
            plt.imshow(img, cmap='gray')
            plt.title('Example Car Image')
            plt.subplot(122)
            plt.imshow(hog_image, cmap='gray')
            plt.title('HOG Visualization')
        return features, hog_image
    # Otherwise call with one output
    else:      
        features = hog(img, orientations=orient, 
                       pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block), 
                       transform_sqrt=False, 
                       visualise=vis, feature_vector=feature_vec)
        return features


    
# Method to combine all three sptial binning, histogram of color, and HOG features    
def extract_features(imgs, cspace='RGB', spatial_size=(32, 32),
                        hist_bins=32, hist_range=(0, 256),orient=9, 
                        pix_per_cell=8, cell_per_block=2, hog_channel=0):
    """
    Method to convert image to different color space and combine hog feature, histogram 
    of color features, and spatial bin color features into a single feature vector
    """
    
    # Create a list to append feature vectors
    features = []
    for file in imgs:
        image = mpimg.imread(file)
        if cspace != 'RGB':
            if cspace == 'HSV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
            elif cspace == 'LUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2LUV)
            elif cspace == 'HLS':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
            elif cspace == 'YUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
            elif cspace == 'YCRCB':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YCrCb)
        else: feature_image = np.copy(image)
        if hog_channel == 'ALL':
            hog_features = []
            for channel in range(feature_image.shape[2]):
                hog_features.append(get_hog_features(feature_image[:,:,channel], 
                                    orient, pix_per_cell, cell_per_block, 
                                    vis=False, feature_vec=True))
            hog_features = np.ravel(hog_features)        
        else:
            hog_features = get_hog_features(feature_image[:,:,hog_channel], orient, 
                        pix_per_cell, cell_per_block, vis=False, feature_vec=True)
        hist_features = color_hist(feature_image, nbins=hist_bins, bins_range = hist_range)
        spatial_features = bin_spatial(feature_image, size=spatial_size)
        features.append(np.concatenate((hog_features,spatial_features,hist_features)))
    # Return list of feature vectors
    return features    
    

## Data Exploration

#### (a). Plot car and non-car image

In [ ]:
# Load cars and non-cars dataset
car_files = glob.glob('vehicles/vehicles/*/*.png')
noncar_files = glob.glob('non-vehicles/non-vehicles/*/*.png')
print("Number of car images: ",len(car_files))
print("Number of non-car images: ",len(noncar_files))
# randomize the index of cars and noncars dataset
car_idx = np.random.randint(len(car_files) - 1)
noncar_idx = np.random.randint(len(noncar_files) - 1)

img_car = mpimg.imread(car_files[car_idx])
img_noncar = mpimg.imread(noncar_files[noncar_idx])

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(5,2.5))
ax1.imshow(img_car)
ax1.set_title('Car Image',fontsize=15)
ax2.imshow(img_noncar)
ax2.set_title('Non-car Image', fontsize=15)


#### (b). Visualize Hog, Color Histogram and Spatial Binned Features for a car image

In [ ]:
# Just create a HOG parameters to display the HOG image/features
colorspace = 'YCRCB' # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
orient = 9
pix_per_cell = 8
cell_per_block = 2
hog_channel = 0 # Can be 0, 1, 2, or "ALL"
spatial_feat = True  # Spatial features on or off
hist_feat = True  # Histogram features on or off
hog_feat = True  # HOG features on or off

In [ ]:
# spatial binning of car image
bin_features = bin_spatial(img_car, size=(32, 32), print_flag = 'yes')

HOG visualization below with gradient directions are shown.  As you can see the shape of the car is almost standout in the HOG visualization plot. 

In [ ]:
# HOG features
img_car = img_car[:,:,0]
hog_car = get_hog_features(img_car, orient=orient, pix_per_cell=pix_per_cell, 
                                cell_per_block= cell_per_block, vis=True, feature_vec=False, 
                                print_flag='yes')

#### (c). Visualize combined and normalized features for a car and non-car image
Below is the combined feature vectors of the car and non-car before normalization and after normalization.

In [ ]:
car_features = extract_features(car_files, cspace=colorspace,
                                orient=orient, pix_per_cell=pix_per_cell,
                                cell_per_block=cell_per_block,
                                hog_channel=hog_channel)
print ('Car samples: ', len(car_features))
notcar_features = extract_features(noncar_files, cspace=colorspace,
                                   orient=orient, pix_per_cell=pix_per_cell,
                                   cell_per_block=cell_per_block,
                                   hog_channel=hog_channel)
print ('Notcar samples: ', len(notcar_features))

In [ ]:
# Create an array stack of feature vectors
x = np.vstack((car_features, notcar_features)).astype(np.float64)
# Fit a per-column scaler
x_scaler = StandardScaler().fit(x)
# Apply the scaler to X
scaled_x = x_scaler.transform(x)
# Plot an example of raw and scaled features
fig = plt.figure(figsize=(12, 4))
plt.subplot(131)
plt.imshow(img_car, cmap='gray')
plt.title('Original Image')
plt.subplot(132)
print('car index: ', car_idx)
plt.plot(x[car_idx])
plt.title('Raw Features')
plt.subplot(133)
plt.plot(scaled_x[car_idx])
plt.title('Normalized Features')
fig.tight_layout()
plt.show()

## SVM Training

In [ ]:
#Final parameters 
colorspace = 'YCRCB' # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
orient = 9
pix_per_cell = 8
cell_per_block = 2
hog_channel = "ALL" # Can be 0, 1, 2, or "ALL"
spatial_size = (32,32)
hist_range = (0,256)
hist_bins = 32


def svm (car_files,noncar_files,cspace=colorspace,spatial_size=spatial_size,hist_bins=hist_bins, 
         hist_range=hist_range,orient=orient,pix_per_cell=pix_per_cell, cell_per_block=cell_per_block, 
         hog_channel=hog_channel):
    """
    Stack car and non-car feature vectors and normalize them before fitting into a Linear SVM.
    Give SVM prediction and compute time for both SVM train and prediction.
    """
    
    car_features = extract_features(car_files, cspace=colorspace, spatial_size=spatial_size,
                        hist_bins=hist_bins, hist_range=hist_range,orient=orient, 
                        pix_per_cell=pix_per_cell, cell_per_block=cell_per_block, hog_channel=hog_channel)
    noncar_features = extract_features(noncar_files, cspace=colorspace, spatial_size=spatial_size,
                        hist_bins=hist_bins, hist_range=hist_range,orient=orient, 
                        pix_per_cell=pix_per_cell, cell_per_block=cell_per_block, hog_channel=hog_channel)

    # Create an array stack of feature vectors
    X = np.vstack((car_features, noncar_features)).astype(np.float64)                        
    # Fit a per-column scaler
    X_scaler = StandardScaler().fit(X)
    # Apply the scaler to X
    scaled_X = X_scaler.transform(X)

    # Define the labels vector
    y = np.hstack((np.ones(len(car_features)), np.zeros(len(noncar_features))))

    # Split up data into randomized training and test sets
    rand_state = np.random.randint(0, 100)
    X_train, X_test, y_train, y_test = train_test_split(scaled_X, y, test_size=0.2, random_state=rand_state)

    
    print('Using:',orient,'orientations',pix_per_cell,
        'pixels per cell and', cell_per_block,'cells per block')
    print('Feature vector length:', len(X_train[0]))
    
    # Use a linear SVC 
    svc = LinearSVC()
    
    # Training for the SVC
    t=time.time()
    svc.fit(X_train, y_train)
    t2 = time.time()
    print(round(t2-t, 2), 'Seconds to train SVC...')
    
    # Check the score of the SVC
    print('Test Accuracy of SVC = ', round(svc.score(X_test, y_test), 4))
    
    # Check the prediction time for a single sample
    t=time.time()
    n_predict = 10
    print('My SVC predicts: ', svc.predict(X_test[0:n_predict]))
    print('For these',n_predict, 'labels: ', y_test[0:n_predict])
    t2 = time.time()
    print(round(t2-t, 5), 'Seconds to predict', n_predict,'labels with SVC')
    
    #save svc model
    pickle_svc = {}
    pickle_svc['X_scaler'] = X_scaler
    pickle_svc['svc'] = svc
    pickle.dump(pickle_svc, open('pickle_svc.p', 'wb'))


In [ ]:
svm_train = svm(car_files,noncar_files,cspace=colorspace,spatial_size=spatial_size,hist_bins=hist_bins, 
         hist_range=hist_range,orient=orient,pix_per_cell=pix_per_cell, cell_per_block=cell_per_block, 
         hog_channel=hog_channel)

## Image Processing


#### Helper Functions for Image Processing


In [ ]:
def convert_color(img, conv='RGB2YCrCb'):
    """
    color conversion for a image to different color space
    """
    
    if conv == 'RGB2YCrCb':
        return cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
    if conv == 'BGR2YCrCb':
        return cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)
    if conv == 'RGB2LUV':
        return cv2.cvtColor(img, cv2.COLOR_RGB2LUV)

    
def add_heat(heatmap, bbox_list):
    """
    Add one to the heatmap for every bounding box.
    """
    
    # Iterate through list of bboxes
    for box in bbox_list:
        # Add += 1 for all pixels inside each bbox
        # Assuming each "box" takes the form ((x1, y1), (x2, y2))
        heatmap[box[0][1]:box[1][1], box[0][0]:box[1][0]] += 1
    # Return updated heatmap
    return heatmap# Iterate through list of bboxes
    
def apply_threshold(heatmap, threshold):
    """
    Apply heatmap threshold.  Zero out pixels if below the threshold.
    """
    
    # Zero out pixels below the threshold
    heatmap[heatmap <= threshold] = 0
    # Return thresholded map
    return heatmap

def draw_labeled_bboxes(img, labels):
    """
    Base on the image labels, draw a bounding box around objects.
    """
    
    # Iterate through all detected cars
    for car_number in range(1, labels[1]+1):
        # Find pixels with each car_number label value
        nonzero = (labels[0] == car_number).nonzero()
        # Identify x and y values of those pixels
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        # Define a bounding box based on min/max x and y
        bbox = ((np.min(nonzerox), np.min(nonzeroy)), (np.max(nonzerox), np.max(nonzeroy)))
        # Draw the box on the image
        cv2.rectangle(img, bbox[0], bbox[1], (255,0,0), 3)
    # Return the image
    return img

#### Function that extract cars in the image

In [ ]:
# Function that can extract features using hog sub-sampling
def find_cars(img, ystart, ystop, scale, X_scaler, svc, orient, pix_per_cell, cell_per_block, 
              spatial_size, hist_bins, print_flag='no'):
    """
    This is the main pipeline to find a car.
    1. Define a ROI region to search for cars on the image.
    2. Scale image accordingly if scale is defined different than 1
    3. Split the image to separate channels
    4. Compute the hog features for the 3 image channels
    5. Define numbers of blocks, steps and window. For every image patch, compute the hog features
       for individual channel, stack them up, and combine with other color features.
    6. Predict the final feature using pre-define svm.
    7. Draw a rectangle bounding box around predicted features on image
    8. Return draw image and list of bounding box.
    """
    
    draw_img = np.copy(img)
    img = img.astype(np.float32)/255
    
    img_tosearch = img[ystart:ystop,:,:]
    if (print_flag == 'yes'):
        plt.title('ROI to search for car')
        plt.imshow(img_tosearch)
    img_tosearch = convert_color(img_tosearch, conv='RGB2YCrCb')
    
    if scale != 1:
        img_shape = img_tosearch.shape
        img_tosearch = cv2.resize(img_tosearch, (np.int(img_shape[1]/scale), np.int(img_shape[0]/scale)))
#         plt.title('ROI to search for car after applying scale')
#         plt.imshow(img_tosearch)
        
    ch1 = img_tosearch[:,:,0]
    ch2 = img_tosearch[:,:,1]
    ch3 = img_tosearch[:,:,2]

    # Define blocks and steps
    nxblocks = (ch1.shape[1] // pix_per_cell)-1
    nyblocks = (ch1.shape[0] // pix_per_cell)-1 
    nfeat_per_block = orient*cell_per_block**2

    # 64 was the orginal sampling rate, with 8 cells and 8 pix per cell
    window = 64
    nblocks_per_window = (window // pix_per_cell)-1 
    cells_per_step = 2  # Instead of overlap, define how many cells to step
    nxsteps = (nxblocks - nblocks_per_window) // cells_per_step
    nysteps = (nyblocks - nblocks_per_window) // cells_per_step
    
    # Compute individual channel HOG features for the entire image
    hog1 = get_hog_features(ch1, orient, pix_per_cell, cell_per_block, feature_vec=False)
    hog2 = get_hog_features(ch2, orient, pix_per_cell, cell_per_block, feature_vec=False)
    hog3 = get_hog_features(ch3, orient, pix_per_cell, cell_per_block, feature_vec=False)
    
    bbox_list=[]
    for xb in range(nxsteps):
        for yb in range(nysteps):
            ypos = yb*cells_per_step
            xpos = xb*cells_per_step
            # Extract HOG for this patch
            hog_feat1 = hog1[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel() 
            hog_feat2 = hog2[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel() 
            hog_feat3 = hog3[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel() 
            hog_features = np.hstack((hog_feat1, hog_feat2, hog_feat3))

            xleft = xpos*pix_per_cell
            ytop = ypos*pix_per_cell

            # Extract the image patch
            subimg = cv2.resize(img_tosearch[ytop:ytop+window, xleft:xleft+window], (64,64))
          
            # Get color features
            spatial_features = bin_spatial(subimg, size=spatial_size)
            hist_features = color_hist(subimg, nbins=hist_bins)

            # Scale features and make a prediction
            test_features = X_scaler.transform(np.hstack((hog_features, spatial_features, hist_features)).reshape(1, -1))    
            #test_features = X_scaler.transform(np.hstack((shape_feat, hist_feat)).reshape(1, -1))    
            test_prediction = svc.predict(test_features)
            
            if test_prediction == 1:
                xbox_left = np.int(xleft*scale)
                ytop_draw = np.int(ytop*scale)
                win_draw = np.int(window*scale)
                cv2.rectangle(draw_img,(xbox_left, ytop_draw+ystart),(xbox_left+win_draw,ytop_draw+win_draw+ystart),(0,0,255),3) 
                bbox_list.append(((xbox_left, ytop_draw+ystart),(xbox_left+win_draw,ytop_draw+win_draw+ystart)))
                
    return draw_img,bbox_list

#### Test with six test images


In [ ]:
orient = 9
pix_per_cell = 8
cell_per_block = 2
spatial_size = (32,32)
hist_range = (0,256)
hist_bins = 32

ystart = 400
ystop = 656
scales = [1.5]

threshold = 1.5


#restore svc model and X_scaler
pickle_svc = pickle.load(open('pickle_svc.p', 'rb'))
svc = pickle_svc['svc']
X_scaler = pickle_svc['X_scaler']

test_files = glob.glob('test_images/*.jpg')


for test_file in test_files:
    test_img = mpimg.imread(test_file)
    heat = np.zeros_like(test_img[:,:,0]).astype(np.float)
    
    curr_img = np.copy(test_img)
    for curr_scale in scales:
        curr_img,box_list = find_cars(curr_img, ystart, ystop, curr_scale, X_scaler, svc, 
                                      orient, pix_per_cell, cell_per_block, spatial_size, 
                                      hist_bins, print_flag='no')
        heat = add_heat(heat,box_list)
        heat = apply_threshold(heat,threshold)
        heatmap = np.clip(heat, 0, 255)
    
    # Find final boxes from heatmap using label function
    labels = label(heatmap)
    draw_image = draw_labeled_bboxes(np.copy(test_img), labels)
    
    f, (ax1, ax2, ax3) = plt.subplots(1,3,figsize=(10,5))
    
    ax1.imshow(curr_img)
    ax1.set_title('window search')
    
    ax2.imshow(heatmap, cmap='hot')
    ax2.set_title('heatmap')
    
    ax3.imshow(draw_image)
    ax3.set_title('bounding box')

In [ ]:
for test_file in test_files:
    test_img = mpimg.imread(test_file)
    _,_ = find_cars(test_img, ystart, ystop, curr_scale, X_scaler, svc, 
                                      orient, pix_per_cell, cell_per_block, spatial_size, 
                                      hist_bins, print_flag='yes')

## Video Processing


In [ ]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML

#restore svc model and X_scaler
pickle_svc = pickle.load(open('pickle_svc.p', 'rb'))
svc = pickle_svc['svc']
X_scaler = pickle_svc['X_scaler']
orient = 9
pix_per_cell = 8
cell_per_block = 2
spatial_size = (32,32)
hist_range = (0,256)
hist_bins = 32
ystart = 400
ystop = 656
scales = [1.5]
threshold = 5
num_frames = 5   # number of frame in the queue



class HotWindows():
    """
    Keep track of n previous hot windows
    Compute cumulative heat map over time

    self.windows is a queue of lists of bounding boxes.
    The list can be of arbitrary size.
    
    Each element in the queue represents the list of
    bounding boxes at a particular time frame.
    """
    def __init__(self, n):
        """
        constructor of the HotWindows class.
        """
        self.n = n
        self.windows = []  

    def add_windows(self, new_windows):
        """
        Push new windows to queue
        Pop from queue if it is full
        """
        self.windows.append(new_windows)

        queue_full = len(self.windows) >= self.n
        if queue_full:
            _ = self.windows.pop(0)

    def get_windows(self):
        """
        Concatenate all lists in the queue and return the list
        """
        out_windows = []
        for window in self.windows:
            out_windows = out_windows + window
        return out_windows
    
    
hot_windows = HotWindows(num_frames)

def process_image(img):
    """
    Find the hot windows. Add the new windows from the new frame into the list
    until the queue is full.
    
    Compute the heatmap base on hot windows, apply the heatmap threshold and find the labels 
    Finally, draw the bounding box around objects.
    
    """
    global hot_windows

    heat = np.zeros_like(img[:,:,0]).astype(np.float)
    curr_img = np.copy(img)
    for curr_scale in scales:
        output_img,box_list = find_cars(curr_img, ystart, ystop, curr_scale, X_scaler, svc, orient, pix_per_cell, cell_per_block, spatial_size, hist_bins)
        # Add new hot windows to HotWindows queue
        hot_windows.add_windows(box_list)
        all_hot_windows = hot_windows.get_windows()

        # Calculate and draw heat map
        heat = add_heat(heat, all_hot_windows)
        heat = apply_threshold(heat, threshold)
        labels = label(heat)

        # Draw final bounding boxes
        draw_img = draw_labeled_bboxes(np.copy(img), labels)
        #draw_img = draw_labeled_bboxes(output_img, labels)
    return draw_img

In [ ]:
white_output = 'test_video_output.mp4'
clip1 = VideoFileClip("test_video.mp4")
white_clip = clip1.fl_image(process_image)
%time white_clip.write_videofile(white_output, audio=False)

In [ ]:
white_output = 'project_video_output.mp4'
clip1 = VideoFileClip("project_video.mp4")
white_clip = clip1.fl_image(process_image)
%time white_clip.write_videofile(white_output, audio=False)